## Shell calls from Python on Windows

Quick tour of invoking classic `cmd.exe` commands, capturing their output, and sending that output to files.

In [1]:
import subprocess
from pathlib import Path

# Base working directory for all shell calls
workdir = Path.cwd()
print(f"Working in: {workdir}")

Working in: c:\Users\zepedro\Documents\GitHub\PMNEC


In [ ]:
# Simple call: ask cmd for the current directory
subprocess.run([
    "cmd",
    "/c", # Use /c to run command and exit. Without it, cmd stays open waiting for input
    "cd"
], cwd=workdir)

CompletedProcess(args=['cmd', '/c', 'cd'], returncode=0)

In [ ]:
# Another example: run cmd interactively but terminate it after a short delay
import subprocess
import time
import pathlib

workdir = pathlib.Path.cwd()
p = subprocess.Popen(["cmd", "cd"], cwd=workdir)
time.sleep(1)  # Let it run briefly
p.terminate()  # Close the subprocess
p.wait()  # Wait for it to finish
print("Subprocess terminated")

Subprocess terminated


In [3]:
# Capture output to a Python variable (list directory entries)
result = subprocess.run([
    "cmd",
    "/c",
    "dir /b"
], capture_output=True, text=True, cwd=workdir)

lines = result.stdout.splitlines()
print("Exit code:", result.returncode)
print("First 5 entries:", lines[:5])
print("stderr:", result.stderr)

Exit code: 0
First 5 entries: ['.gitattributes', '.gitignore', '00.env_and_basics.ipynb', '01.text_interaction.ipynb', '02.context_mangers.ipynb']
stderr: 


In [4]:
# Produce output as a file via cmd and read it back in Python
outfile = workdir / "shell_output.txt"
subprocess.run([
    "cmd",
    "/c",
    f"echo %DATE% %TIME% > {outfile}"
], check=True, cwd=workdir)

print("File created:", outfile)
print("Contents:\n", outfile.read_text())

File created: c:\Users\zepedro\Documents\GitHub\PMNEC\shell_output.txt
Contents:
 12/12/2025  8:27:59.54 



In [13]:
# Direct stdout and stderr to separate files via subprocess file handles
stdout_file = workdir / "stdout.txt"
stderr_file = workdir / "stderr.txt"

# Command that produces both output and error
with open(stdout_file, 'w') as outf, open(stderr_file, 'w') as errf:
    subprocess.run([
        "cmd",
        "/c",
        "dir & haha"
    ], stdout=outf, stderr=errf, cwd=workdir)

print("Stdout file contents:\n", stdout_file.read_text())
print("Stderr file contents:\n", stderr_file.read_text())

Stdout file contents:
  Volume in drive C has no label.
 Volume Serial Number is E075-62B0

 Directory of c:\Users\zepedro\Documents\GitHub\PMNEC

12/12/2025  08:23 am    <DIR>          .
27/11/2025  02:42 pm    <DIR>          ..
27/11/2025  02:42 pm                66 .gitattributes
27/11/2025  02:42 pm             3,910 .gitignore
27/11/2025  06:02 pm             1,481 00.env_and_basics.ipynb
12/12/2025  06:27 am             7,934 01.text_interaction.ipynb
12/12/2025  06:27 am           496,464 02.context_mangers.ipynb
12/12/2025  06:51 am            10,144 03.temporary_resources.ipynb
12/12/2025  08:13 am            11,369 04.parallel.ipynb
12/12/2025  08:29 am             7,122 05.shell.ipynb
27/11/2025  07:15 pm           297,298 0x.pinn_backwater.ipynb
27/11/2025  06:12 pm           395,842 0x.pinn_Burgers.ipynb
12/12/2025  07:56 am            13,420 a_programming.ipynb
03/12/2025  01:46 pm             5,983 b_programming.ipynb
27/11/2025  05:50 pm            77,719 initial_bounda

In [14]:
# Capture stdout and stderr to buffers via subprocess PIPE
import io

# Command that produces both output and error
result = subprocess.run([
    "cmd",
    "/c",
    "dir & haha"
], capture_output=True, text=True, cwd=workdir)

stdout_buffer = io.StringIO(result.stdout)
stderr_buffer = io.StringIO(result.stderr)

print("Stdout buffer contents:\n", repr(stdout_buffer.getvalue()))
print("Stderr buffer contents:\n", repr(stderr_buffer.getvalue()))

Stdout buffer contents:
 ' Volume in drive C has no label.\n Volume Serial Number is E075-62B0\n\n Directory of c:\\Users\\zepedro\\Documents\\GitHub\\PMNEC\n\n12/12/2025  08:23 am    <DIR>          .\n27/11/2025  02:42 pm    <DIR>          ..\n27/11/2025  02:42 pm                66 .gitattributes\n27/11/2025  02:42 pm             3,910 .gitignore\n27/11/2025  06:02 pm             1,481 00.env_and_basics.ipynb\n12/12/2025  06:27 am             7,934 01.text_interaction.ipynb\n12/12/2025  06:27 am           496,464 02.context_mangers.ipynb\n12/12/2025  06:51 am            10,144 03.temporary_resources.ipynb\n12/12/2025  08:13 am            11,369 04.parallel.ipynb\n12/12/2025  08:29 am             7,122 05.shell.ipynb\n27/11/2025  07:15 pm           297,298 0x.pinn_backwater.ipynb\n27/11/2025  06:12 pm           395,842 0x.pinn_Burgers.ipynb\n12/12/2025  07:56 am            13,420 a_programming.ipynb\n03/12/2025  01:46 pm             5,983 b_programming.ipynb\n27/11/2025  05:50 pm      